In [84]:
import subprocess
import os
import re

In [85]:
js_getVARS_loc = os.path.join(os.path.abspath('parsergen'), "get_vars.js")
php_getVARS_loc = os.path.join(os.path.abspath('parsergen'), "get_vars.php")
php_samples_loc = os.path.join(os.path.abspath('parsergen'), "php_samples")
js_samples_loc = os.path.join(os.path.abspath('parsergen'), "js_samples")

In [86]:
print(js_getVARS_loc)
print(php_getVARS_loc)
print(php_samples_loc)
print(js_samples_loc)

/home/shanks/CSE_00003_A/bloom-experiment-lab/dataprocessing/parsergen/get_vars.js
/home/shanks/CSE_00003_A/bloom-experiment-lab/dataprocessing/parsergen/get_vars.php
/home/shanks/CSE_00003_A/bloom-experiment-lab/dataprocessing/parsergen/php_samples
/home/shanks/CSE_00003_A/bloom-experiment-lab/dataprocessing/parsergen/js_samples


In [87]:
# Get all files with parent directory
php_samples = [os.path.join(php_samples_loc, f) for f in os.listdir(php_samples_loc) if os.path.isfile(os.path.join(php_samples_loc, f))]
js_samples = [os.path.join(js_samples_loc, f) for f in os.listdir(js_samples_loc) if os.path.isfile(os.path.join(js_samples_loc, f))]

print(php_samples)
print(js_samples)

['/home/shanks/CSE_00003_A/bloom-experiment-lab/dataprocessing/parsergen/php_samples/sample1.php']
['/home/shanks/CSE_00003_A/bloom-experiment-lab/dataprocessing/parsergen/js_samples/sample1.js']


In [88]:
js_get_vars_sourcetype = "module"
node_cmd = ["node", js_getVARS_loc, js_get_vars_sourcetype, js_samples[0]]
php_cmd = ["php", php_getVARS_loc, php_samples[0]]

try:
    node_output = subprocess.check_output(node_cmd, stderr=subprocess.PIPE).decode('utf-8')
    jsvars = node_output.split(',')
    php_output = subprocess.check_output(php_cmd, stderr=subprocess.PIPE).decode('utf-8')
    phpvars = php_output.split(',')
except subprocess.CalledProcessError as e:
    print(e.stderr)


In [89]:
print(jsvars)
print(phpvars)

['express', 'crypto', 'app', 'db', 'FLAG', 'PORT', 'users', 'isAdmin', 'newAdmin', 'undefined', 'query', 'id']
['$input', '$data', '$api_url', '$options', '$context', '$result', "$_POST['login-submit']", "$_POST['username']", "$_POST['password']", '$username', '$password', "$_SERVER['REMOTE_ADDR']"]


In [136]:
def code_cleaner(filename):
    with open(filename, 'r') as f:
        code = f.read()

    # GENERAL: 
    # remove multiline comments single line
    code = re.sub(r'/\*(.*?)\*/', '', code, flags=re.DOTALL)
    # remove all single line comments except for the ones that are in a string like ', ", or `
    code = re.sub(r'(?<!:)(//|#)(?!:).*', '', code)


    # make concatenated strings that uses '+' operator single line
    code = re.sub(r'\+\s*\'', '', code)
    # make concatenated strings that uses '.' operator single line
    code = re.sub(r'\.\s*\'', '', code)
    # split code into lines
    code = code.split('\n')
    # remove leading and trailing whitespace
    code = [line.strip() for line in code]

    # remove all elements that are symbols only in a string
    code = [line for line in code if not re.match(r'^\W+$', line)]

    # PHP:
    # remove all php tags
    code = [line for line in code if not line.startswith('<?php') and not line.startswith('?>')]

    # lastly remove all empty lines
    code = list(filter(None, code))

    return code

js_code = code_cleaner(js_samples[0])
php_code = code_cleaner(php_samples[0])

print(js_code)
print(php_code)



["const express = require('express');", "const crypto = require('crypto');", 'const app = express();', "const db = require('better-sqlite3')('db.sqlite3');", 'db.exec(`DROP TABLE IF EXISTS users;`);', 'db.exec(`CREATE TABLE users(', 'id INTEGER PRIMARY KEY,', 'username TEXT,', 'password TEXT', 'const FLAG = process.env.FLAG || "dice{test_flag}";', 'const PORT = process.env.PORT || 3000;', 'const users = [...Array(100_000)].map(() => ({ user: `user-${crypto.randomUUID()}`, pass: crypto.randomBytes(8).toString("hex") }));', 'db.exec(`INSERT INTO users (id, username, password) VALUES ${users.map((u,i) => `(${i}, \'${u.user}\', \'${u.pass}\')`).join(", ")}`);', 'const isAdmin = {};', 'const newAdmin = users[Math.floor(Math.random() * users.length)];', 'isAdmin[newAdmin.user] = true;', 'app.use(express.urlencoded({ extended: false }));', 'app.use(express.static("public"));', 'app.post("/api/login", (req, res) => {', 'const { user, pass } = req.body;', 'console.log("[REQUEST BODY] : ", req.b